![Bank](http://www.abm.co.uk/banking-finance-services/wp-content/uploads/sites/92/2016/04/Banking_Icons_Frame4c.jpg)

A financial institution is planning to roll out a stock market trading faciliation service for their existing
account holders. This service costs significant amount of money for the bank in terms of infra, licensing
and people cost. To make the serive offering profitable, they charge a percentage base comission on every
trade transaction. However this is not a unique service offered by them, many of their other competitors are
offering the same service and at lesser commission some times. To retain or attract people who trade heavily
on stock market and in turn generate a good commission for institution, they are planning to offer discounts
as they roll out the service to entire customer base.

Problem is , that this discount, hampers profits coming from the customers who do not trade in large
quantities . To tackle this issue , company wants to offer discounts selectively. To be able to do so, they need
to know which of their customers are going to be heavy traders or money makers for them.
To be able to do this, they decided to do a beta run of their service to a small chunk of their customer base
[approx 10000 people]. For these customers they have manually divided them into two revenue categories 1
and 2. Revenue one category is the one which are moeny makers for the bank, revenue category 2 are the
ones which need to be kept out of discount offers.

We need to use this study’s data to build a prediction model which should be able to identify if a customer is
potentially eligible for discounts [falls In revnue grid category 1]. Lets get the data and begin.

In [53]:

rg_train=read.csv("../input/rg_train.csv",stringsAsFactors=F)
rg_test=read.csv("../input/rg_test.csv",stringsAsFactors=F)

## Let's take a sneak peak of our train & test data 
head(rg_train)


REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,⋯,Investment.Tax.Saving.Bond,Home.Loan,Online.Purchase.Amount,Revenue.Grid,gender,region,Investment.in.Commudity,Investment.in.Equity,Investment.in.Derivative,Portfolio.Balance
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
2148,1,45-50,Partner,Professional,Professional,Rent Privately,">=35,000",Yes,Yes,⋯,7.49,2.48,0.00,2,Female,South West,65.87,9.27,30.93,87.48
8099,1,61-65,Partner,Retired,Retired,Own Home,"<12,500, >=10,000",No,No,⋯,0.00,3.99,0.00,2,Female,Unknown,42.46,4.49,26.23,110.73
6611,3,31-35,Partner,Professional,Professional,Own Home,">=35,000",No,No,⋯,0.00,0.00,0.00,2,Male,East Anglia,75.38,0.00,26.66,127.57
1950,Zero,55-60,Partner,Professional,Professional,Own Home,">=35,000",No,No,⋯,2.00,0.00,0.00,2,Female,North West,34.78,6.91,29.24,33.79
10857,2,51-55,Partner,Manual Worker,Manual Worker,Own Home,"<27,500, >=25,000",Yes,Yes,⋯,0.00,0.00,0.00,2,Female,South West,48.58,9.58,20.65,56.17
10853,2,31-35,Partner,Professional,Professional,Own Home,"<25,000, >=22,500",No,No,⋯,52.94,7.46,7.98,1,Male,North,77.47,38.37,85.35,200.23


Variable names are self explanatory as to what they represent. Our train data contains 32 variables like age_band,status,Occupation,home status etc. which will be involved in modelling process. Similarily for our test data given below there are 31 variables excluding that response variable (Revenue Grid) which we will predict.

In [54]:
head(rg_test)

REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,⋯,Investment.in.Mutual.Fund,Investment.Tax.Saving.Bond,Home.Loan,Online.Purchase.Amount,gender,region,Investment.in.Commudity,Investment.in.Equity,Investment.in.Derivative,Portfolio.Balance
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
9,1,36-40,Partner,Professional,Professional,Own Home,">=35,000",No,No,⋯,74.93,41.71,35.41,11.99,Female,Scotland,48.67,60.23,63.58,162.40
13,Zero,61-65,Partner,Retired,Retired,Own Home,"<12,500, >=10,000",No,No,⋯,0.00,0.00,5.99,0.00,Female,Wales,5.00,1.00,0.00,-18.84
18,Zero,26-30,Partner,Unknown,Professional,Own Home,">=35,000",Yes,No,⋯,123.39,0.00,39.96,0.00,Female,Scotland,63.87,36.64,59.96,115.89
21,1,41-45,Partner,Secretarial/Admin,Manual Worker,Own Home,"<12,500, >=10,000",No,No,⋯,86.43,12.49,21.46,0.00,Female,West Midlands,39.48,38.63,62.04,134.44
26,Zero,26-30,Partner,Professional,Professional,Own Home,">=35,000",No,No,⋯,25.48,0.00,0.00,0.00,Female,East Midlands,15.20,5.08,17.74,27.95
28,Zero,71+,Partner,Retired,Retired,Own Home,"<22,500, >=20,000",No,No,⋯,37.47,9.99,0.00,0.00,Female,South West,75.26,56.70,80.93,144.84


In [55]:
library(dplyr)
glimpse(rg_train)

Observations: 8,124
Variables: 32
$ REF_NO                          <int> 2148, 8099, 6611, 1950, 10857, 10853,…
$ children                        <chr> "1", "1", "3", "Zero", "2", "2", "3",…
$ age_band                        <chr> "45-50", "61-65", "31-35", "55-60", "…
$ status                          <chr> "Partner", "Partner", "Partner", "Par…
$ occupation                      <chr> "Professional", "Retired", "Professio…
$ occupation_partner              <chr> "Professional", "Retired", "Professio…
$ home_status                     <chr> "Rent Privately", "Own Home", "Own Ho…
$ family_income                   <chr> ">=35,000", "<12,500, >=10,000", ">=3…
$ self_employed                   <chr> "Yes", "No", "No", "No", "Yes", "No",…
$ self_employed_partner           <chr> "Yes", "No", "No", "No", "Yes", "No",…
$ year_last_moved                 <int> 1999, 1959, 1992, 1990, 1994, 1988, 1…
$ TVarea                          <chr> "HTV", "Unknown", "Anglia", "Granada"…
$ post_code       

In [56]:
glimpse(rg_test)

Observations: 2,031
Variables: 31
$ REF_NO                          <int> 9, 13, 18, 21, 26, 28, 34, 44, 57, 59…
$ children                        <chr> "1", "Zero", "Zero", "1", "Zero", "Ze…
$ age_band                        <chr> "36-40", "61-65", "26-30", "41-45", "…
$ status                          <chr> "Partner", "Partner", "Partner", "Par…
$ occupation                      <chr> "Professional", "Retired", "Unknown",…
$ occupation_partner              <chr> "Professional", "Retired", "Professio…
$ home_status                     <chr> "Own Home", "Own Home", "Own Home", "…
$ family_income                   <chr> ">=35,000", "<12,500, >=10,000", ">=3…
$ self_employed                   <chr> "No", "No", "Yes", "No", "No", "No", …
$ self_employed_partner           <chr> "No", "No", "No", "No", "No", "No", "…
$ year_last_moved                 <int> 1993, 1987, 1998, 1983, 1995, 1972, 1…
$ TVarea                          <chr> "Scottish TV", "Granada", "Scottish T…
$ post_code       

Data Preparation:- 

We’ll combine our two datasets so that we do not need to prepare data separately for them. And we’ll also
avoid problem of dealing with different columns in different datasets.
However before combining them, we’ll need to add response column to test because number of columns need
to be same for two datasets to stack vertically.

In [57]:
rg_test$Revenue.Grid=NA
rg_train$data='train'
rg_test$data='test'
rg=rbind(rg_train,rg_test)

Lets start with looking at our first predictor variable in the data which is
“children”.

In [59]:
table(rg$children)


   1    2    3   4+ Zero 
1848 1607  473   19 6208 

From glimpse function above we saw that the variable 'children' has come as character inspite of being numeric. We can easily convert this, to numeric data without any concern.

In [61]:
rg = rg %>%
mutate(children=ifelse(children=="Zero",0,substr(children,1,1)),
children=as.numeric(children))

### To check whether it has been converted to numeric or not we will run glimpse again
glimpse(rg)

Observations: 10,155
Variables: 33
$ REF_NO                          <int> 2148, 8099, 6611, 1950, 10857, 10853,…
$ children                        <dbl> 1, 1, 3, 0, 2, 2, 3, 1, 2, 3, 3, 2, 1…
$ age_band                        <chr> "45-50", "61-65", "31-35", "55-60", "…
$ status                          <chr> "Partner", "Partner", "Partner", "Par…
$ occupation                      <chr> "Professional", "Retired", "Professio…
$ occupation_partner              <chr> "Professional", "Retired", "Professio…
$ home_status                     <chr> "Rent Privately", "Own Home", "Own Ho…
$ family_income                   <chr> ">=35,000", "<12,500, >=10,000", ">=3…
$ self_employed                   <chr> "Yes", "No", "No", "No", "Yes", "No",…
$ self_employed_partner           <chr> "Yes", "No", "No", "No", "Yes", "No",…
$ year_last_moved                 <int> 1999, 1959, 1992, 1990, 1994, 1988, 1…
$ TVarea                          <chr> "HTV", "Unknown", "Anglia", "Granada"…
$ post_code      

Lets look at age band variable , we can possibly convert this to numeric by taking average of age ranges.
Lets look at the frequency table any way to find if there are any non-numeric fields.

In [62]:
table(rg$age_band)


  18-21   22-25   26-30   31-35   36-40   41-45   45-50   51-55   55-60   61-65 
     63     456     927    1061    1134    1112    1359    1052    1047     881 
  65-70     71+ Unknown 
    598     410      55 

In [63]:
rg=rg %>%
mutate(a1=as.numeric(substr(age_band,1,2)),
a2=as.numeric(substr(age_band,4,5)),
age=ifelse(substr(age_band,1,2)=="71",71,ifelse(age_band=="Unknown",NA,0.5*(a1+a2)))
) %>%
select(-a1,-a2,-age_band)

Warning message:
“NAs introduced by coercion”Warning message:
“NAs introduced by coercion”

In [64]:
glimpse(rg)



Observations: 10,155
Variables: 33
$ REF_NO                          <int> 2148, 8099, 6611, 1950, 10857, 10853,…
$ children                        <dbl> 1, 1, 3, 0, 2, 2, 3, 1, 2, 3, 3, 2, 1…
$ status                          <chr> "Partner", "Partner", "Partner", "Par…
$ occupation                      <chr> "Professional", "Retired", "Professio…
$ occupation_partner              <chr> "Professional", "Retired", "Professio…
$ home_status                     <chr> "Rent Privately", "Own Home", "Own Ho…
$ family_income                   <chr> ">=35,000", "<12,500, >=10,000", ">=3…
$ self_employed                   <chr> "Yes", "No", "No", "No", "Yes", "No",…
$ self_employed_partner           <chr> "Yes", "No", "No", "No", "Yes", "No",…
$ year_last_moved                 <int> 1999, 1959, 1992, 1990, 1994, 1988, 1…
$ TVarea                          <chr> "HTV", "Unknown", "Anglia", "Granada"…
$ post_code                       <chr> "SN4 7EJ", "CH63 8QN", "NR31 9UN", "W…
$ post_area      

Next we’ll be looking at various categorical variables & create dummies for them. Instead of converting one by one, alternatively we can write a function which should ignore categories with low count.  

In [65]:
CreateDummies=function(data,var,freq_cutoff=0){
t=table(data[,var])
t=t[t>freq_cutoff]
t=sort(t)
categories=names(t)[-1]
for( cat in categories){
name=paste(var,cat,sep="_")
name=gsub(" ","",name)
name=gsub("-","_",name)
name=gsub("\\?","Q",name)
name=gsub("<","LT_",name)
name=gsub("\\+","",name)
name=gsub("\\/","_",name)
name=gsub(">","GT_",name)
name=gsub("=","EQ_",name)
name=gsub(",","",name)
data[,name]=as.numeric(data[,var]==cat)
}
data[,var]=NULL
return(data)
}

Let me explain the function CreateDummies which we created above.

t=table(data[,var]) this bit creates a frequency table for the given categorical column. t here is now
simply a table which contains names as categories of the categorical variable and their frequency in the data.

t=t[t>freq_cutoff] this line of code removes those categories from the table which have frequencies below
the frequency cutoff. ( this is a subjective choice)

‘t=sort(t)’ this line simple sorts the remaining table in ascending order

categories=names(t)[-1] since we sorted the table in ascending manner in the previous line, first category
here has least count. In this line of code we are taking out the category names except the first one ( which
has least count), thus making n-1 dummies from the remaining categories.

name=paste(var,cat,sep="_") all the dummy vars that we intend to make, need to have some name. this
line of code creates that name by concatenating variable name with category name with an _.

name=gsub(" ","",name) subsequent lines like these using gsub are essentially cleaning up the name thats
all. Since we dont have any control over what the categories can be, we are removing special characters and
spaces in the code in an automated fashion.

data[,name]=as.numeric(data[,var]==cat) once we have a cleaned up name, this line creates the dummy
var for that particular category.

data[,var]=NULL once we are done creating dummies for the variable using for loop. Variable is removed
from the data in this line.

Following a bit of code which can be used to extract names of all categorical variables in the data. 

In [66]:
names(rg)[sapply(rg,function(x) is.character(x))]

[1] "status"                "occupation"            "occupation_partner"   
 [4] "home_status"           "family_income"         "self_employed"        
 [7] "self_employed_partner" "TVarea"                "post_code"            
[10] "post_area"             "gender"                "region"               
[13] "data"

We are going to ignore column 'data' for obvious reasons and make dummies for rest.

In [67]:
cat_cols=c("status","occupation","occupation_partner","home_status","family_income","self_employed",
"self_employed_partner","TVarea","gender","region")
for(cat in cat_cols){
rg=CreateDummies(rg,cat,100)
}

We are dropping variables post_code, post_area. They take too many distinct values for these variables to
be useful in modeling process.

In [68]:
rg=rg %>%
select(-post_code,-post_area)

We need to convert our Response variable to 1/0

In [69]:
rg$Revenue.Grid=as.numeric(rg$Revenue.Grid==1)

Next we take care of missing values if any in the data. We will run a for loop as below. 

In [70]:
for(col in names(rg)){
if(sum(is.na(rg[,col]))>0 & !(col %in% c("data","Revenue.Grid"))){
rg[is.na(rg[,col]),col]=mean(rg[rg$data=='train',col],na.rm=T)
}
}

Now lets separate our datasets to begin modeling process

In [71]:
rg_train=rg %>% filter(data=='train') %>% select(-data)
rg_test=rg %>% filter(data=='test') %>% select (-data,-Revenue.Grid)

## Now our train data contains 77 variables instead of 32. 
glimpse(rg_train)

Observations: 8,124
Variables: 77
$ REF_NO                               <int> 2148, 8099, 6611, 1950, 10857, 1…
$ children                             <dbl> 1, 1, 3, 0, 2, 2, 3, 1, 2, 3, 3,…
$ year_last_moved                      <int> 1999, 1959, 1992, 1990, 1994, 19…
$ Average.Credit.Card.Transaction      <dbl> 0.00, 0.00, 119.98, 0.00, 0.00, …
$ Balance.Transfer                     <dbl> 0.00, 77.89, 0.00, 39.99, 161.47…
$ Term.Deposit                         <dbl> 196.95, 0.00, 96.94, 0.00, 14.99…
$ Life.Insurance                       <dbl> 132.42, 134.39, 0.00, 133.93, 58…
$ Medical.Insurance                    <dbl> 0.00, 0.00, 159.97, 0.00, 7.49, …
$ Average.A.C.Balance                  <dbl> 0.00, 7.99, 0.00, 39.48, 57.46, …
$ Personal.Loan                        <dbl> 21.47, 14.98, 0.00, 0.00, 0.00, …
$ Investment.in.Mutual.Fund            <dbl> 24.18, 0.00, 0.00, 0.00, 0.00, 4…
$ Investment.Tax.Saving.Bond           <dbl> 7.49, 0.00, 0.00, 2.00, 0.00, 52…
$ Home.Loan       

In [72]:
## Similarily for test data, variables got increased from 31 to 76 because of inclusion of dummy variables. 

glimpse(rg_test)

Observations: 2,031
Variables: 76
$ REF_NO                               <int> 9, 13, 18, 21, 26, 28, 34, 44, 5…
$ children                             <dbl> 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0,…
$ year_last_moved                      <int> 1993, 1987, 1998, 1983, 1995, 19…
$ Average.Credit.Card.Transaction      <dbl> 0.00, 0.00, 0.00, 35.48, 0.00, 5…
$ Balance.Transfer                     <dbl> 0.01, 24.99, 99.96, 0.00, 0.02, …
$ Term.Deposit                         <dbl> 175.88, 0.00, 39.48, 0.00, 0.00,…
$ Life.Insurance                       <dbl> 67.47, 0.00, 135.41, 122.45, 75.…
$ Medical.Insurance                    <dbl> 0.00, 0.00, 44.48, 39.47, 0.00, …
$ Average.A.C.Balance                  <dbl> 110.95, 0.00, 56.46, 47.45, 4.99…
$ Personal.Loan                        <dbl> 86.39, 0.00, 0.00, 63.93, 0.00, …
$ Investment.in.Mutual.Fund            <dbl> 74.93, 0.00, 123.39, 86.43, 25.4…
$ Investment.Tax.Saving.Bond           <dbl> 41.71, 0.00, 0.00, 12.49, 0.00, …
$ Home.Loan       

If we want to look at tentative performance measure, we’ll break our data into two parts. 

In [73]:
set.seed(2)
s=sample(1:nrow(rg_train),0.8*nrow(rg_train))
rg_train1=rg_train[s,]
rg_train2=rg_train[-s,]

First thing that we’ll be looking to eliminate is severe cases of multi-collinearity. . To examine VIF, we can run a linear regression. We are not concerned with the output of this linear regression model, we are only interested in VIF values of the predictor.

In [74]:
library(car)
for_vif=lm(Revenue.Grid~.-REF_NO,data=rg_train1)
sort(vif(for_vif),decreasing = T)[1:3]

Investment.in.Commudity Investment.in.Derivative     Investment.in.Equity 
               256352539                227034416                157553981

There are few cases of insanely high VIF values , lets eliminate those variables one by one. Code
given below is result of multiple iterations. 

In [75]:
for_vif=lm(Revenue.Grid~.-REF_NO-Investment.in.Commudity
-Investment.in.Derivative-Investment.in.Equity
-region_SouthEast-TVarea_Central-occupation_Professional
-family_income_GT_EQ_35000-region_Scotland
-Portfolio.Balance,
data=rg_train1)
sort(vif(for_vif),decreasing = T)[1:3]

occupation_partner_Unknown occupation_partner_Retired 
                  7.318817                   6.622036 
            region_Unknown 
                  6.379158

All VIF values now are less than 10. This is good enough for logistic regression , Lets move to build our
classification model now. 

In [76]:
log_fit=glm(Revenue.Grid~.-REF_NO-Investment.in.Commudity
-Investment.in.Derivative-Investment.in.Equity
-region_SouthEast-TVarea_Central-occupation_Professional
-family_income_GT_EQ_35000-region_Scotland-Portfolio.Balance,data=rg_train1,family = "binomial")

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

In [ ]:
log_fit=step(log_fit)

Start:  AIC=2019.14
Revenue.Grid ~ (REF_NO + children + year_last_moved + Average.Credit.Card.Transaction + 
    Balance.Transfer + Term.Deposit + Life.Insurance + Medical.Insurance + 
    Average.A.C.Balance + Personal.Loan + Investment.in.Mutual.Fund + 
    Investment.Tax.Saving.Bond + Home.Loan + Online.Purchase.Amount + 
    Investment.in.Commudity + Investment.in.Equity + Investment.in.Derivative + 
    Portfolio.Balance + age + status_Divorced_Separated + status_Single_NeverMarried + 
    status_Partner + occupation_ManualWorker + occupation_Unknown + 
    occupation_BusinessManager + occupation_Housewife + occupation_Secretarial_Admin + 
    occupation_Retired + occupation_Professional + occupation_partner_Housewife + 
    occupation_partner_Secretarial_Admin + occupation_partner_BusinessManager + 
    occupation_partner_ManualWorker + occupation_partner_Retired + 
    occupation_partner_Professional + occupation_partner_Unknown + 
    home_status_RentPrivately + home_status_Ren

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning mess

                                       Df Deviance    AIC
- TVarea_Granada                        1   1885.1 2017.1
- region_Wales                          1   1885.1 2017.1
- home_status_OwnHome                   1   1885.1 2017.1
- region_NorthWest                      1   1885.1 2017.1
- TVarea_Yorkshire                      1   1885.1 2017.1
- region_Unknown                        1   1885.1 2017.1
- occupation_ManualWorker               1   1885.1 2017.1
- home_status_RentfromCouncil_HA        1   1885.2 2017.2
- region_EastMidlands                   1   1885.2 2017.2
- family_income_LT_8000GT_EQ_4000       1   1885.2 2017.2
- TVarea_TVSouthWest                    1   1885.2 2017.2
- age                                   1   1885.2 2017.2
- occupation_partner_Secretarial_Admin  1   1885.2 2017.2
- family_income_LT_17500GT_EQ_15000     1   1885.2 2017.2
- region_SouthWest                      1   1885.3 2017.3
- self_employed_No                      1   1885.3 2017.3
- occupation_p

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


Step:  AIC=2017.14
Revenue.Grid ~ children + year_last_moved + Average.Credit.Card.Transaction + 
    Balance.Transfer + Term.Deposit + Life.Insurance + Medical.Insurance + 
    Average.A.C.Balance + Personal.Loan + Investment.in.Mutual.Fund + 
    Investment.Tax.Saving.Bond + Home.Loan + Online.Purchase.Amount + 
    age + status_Divorced_Separated + status_Single_NeverMarried + 
    status_Partner + occupation_ManualWorker + occupation_Unknown + 
    occupation_BusinessManager + occupation_Housewife + occupation_Secretarial_Admin + 
    occupation_Retired + occupation_partner_Housewife + occupation_partner_Secretarial_Admin + 
    occupation_partner_BusinessManager + occupation_partner_ManualWorker + 
    occupation_partner_Retired + occupation_partner_Professional + 
    occupation_partner_Unknown + home_status_RentPrivately + 
    home_status_RentfromCouncil_HA + home_status_OwnHome + family_income_LT_4000 + 
    family_income_LT_8000GT_EQ_4000 + family_income_LT_10000GT_EQ_8000 +

...

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning mess

                                       Df Deviance    AIC
- occupation_partner_Secretarial_Admin  1   1885.2 2001.2
- age                                   1   1885.2 2001.2
- TVarea_TVSouthWest                    1   1885.2 2001.2
- home_status_RentfromCouncil_HA        1   1885.3 2001.3
- region_EastMidlands                   1   1885.3 2001.3
- self_employed_No                      1   1885.3 2001.3
- occupation_partner_Professional       1   1885.3 2001.3
- family_income_LT_15000GT_EQ_12500     1   1885.3 2001.3
- family_income_LT_17500GT_EQ_15000     1   1885.3 2001.3
- region_SouthWest                      1   1885.4 2001.4
- family_income_LT_4000                 1   1885.4 2001.4
- occupation_partner_Unknown            1   1885.4 2001.4
- occupation_partner_ManualWorker       1   1885.5 2001.5
- TVarea_TyneTees                       1   1885.5 2001.5
- family_income_LT_27500GT_EQ_25000     1   1885.5 2001.5
- occupation_Retired                    1   1885.5 2001.5
- family_incom

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


Step:  AIC=2001.24
Revenue.Grid ~ children + year_last_moved + Average.Credit.Card.Transaction + 
    Balance.Transfer + Term.Deposit + Life.Insurance + Medical.Insurance + 
    Average.A.C.Balance + Personal.Loan + Investment.in.Mutual.Fund + 
    Investment.Tax.Saving.Bond + Home.Loan + Online.Purchase.Amount + 
    age + status_Divorced_Separated + status_Single_NeverMarried + 
    status_Partner + occupation_Unknown + occupation_BusinessManager + 
    occupation_Housewife + occupation_Secretarial_Admin + occupation_Retired + 
    occupation_partner_Housewife + occupation_partner_BusinessManager + 
    occupation_partner_ManualWorker + occupation_partner_Retired + 
    occupation_partner_Professional + occupation_partner_Unknown + 
    home_status_RentPrivately + home_status_RentfromCouncil_HA + 
    family_income_LT_4000 + family_income_LT_10000GT_EQ_8000 + 
    family_income_LT_22500GT_EQ_20000 + family_income_LT_15000GT_EQ_12500 + 
    family_income_LT_17500GT_EQ_15000 + family_

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning mess

In [ ]:
summary(log_fit)

## If we look at summary(log_fit), we’ll find there are still some variable with high p-values.

We will run our logistic regression model with variables selected by step
function and now drop variabe based on p-values on our own from the remaining bunch.

In [ ]:
formula(log_fit)

We can use this to now run our model and drop variables based on p-values too. Code given below is result
of multiple iteration. We have considered 0.1 as p-value cutoff, you can make it lower and drop more variables

In [ ]:
log_fit=glm(Revenue.Grid ~ Average.Credit.Card.Transaction + Balance.Transfer +
Term.Deposit + Life.Insurance + Medical.Insurance + Average.A.C.Balance +
Personal.Loan + Investment.in.Mutual.Fund + Investment.Tax.Saving.Bond +
Home.Loan + Online.Purchase.Amount +
family_income_LT_30000GT_EQ_27500 +
self_employed_partner_No + TVarea_ScottishTV + TVarea_Meridian ,
data=rg_train,family='binomial')
summary(log_fit)

Lets see performance of score model on validation data that we kept aside.

In [ ]:
## We will be using library pROC
library(pROC)

In [ ]:
val.score=predict(log_fit,newdata = rg_train2,type='response')
auc_score=auc(roc(rg_train2$Revenue.Grid,val.score))
auc_score

The area under the curve is 0.96. So the tentative score performance of logistic regression is going to be around 0.96. Lets visualise how is our eventual binary response is behaving w.r.t. score that we obtained. 

In [ ]:
library(ggplot2)
mydata=data.frame(Revenue.Grid=rg_train2$Revenue.Grid,val.score=val.score)
ggplot(mydata,aes(y=Revenue.Grid,x=val.score,color=factor(Revenue.Grid)))+
geom_point()+geom_jitter()

We can see that response 0 is bunched around low scores and response 1 is bunched around high scores,
However there is overlap as well across score values.

We know the tentative performance of logistic regression model in terms of auc score. Next we’ll build the
model on entire training data following the similar steps

In [ ]:
for_vif=lm(Revenue.Grid~.-REF_NO-Investment.in.Commudity
-Investment.in.Derivative-Investment.in.Equity
-region_SouthEast-TVarea_Central-occupation_Professional
-family_income_GT_EQ_35000-region_Scotland-Portfolio.Balance
,data=rg_train)
sort(vif(for_vif),decreasing = T)[1:3]

In [ ]:
log.fit.final=glm(Revenue.Grid~.-REF_NO-Investment.in.Commudity
-Investment.in.Derivative-Investment.in.Equity
-region_SouthEast-TVarea_Central-occupation_Professional
-family_income_GT_EQ_35000-region_Scotland-Portfolio.Balance,
data=rg_train,family='binomial')

In [ ]:
log.fit.final=step(log.fit.final)

In [ ]:
log.fit.final=glm(Revenue.Grid ~ Average.Credit.Card.Transaction + Balance.Transfer +
Term.Deposit + Life.Insurance + Medical.Insurance + Average.A.C.Balance +
Personal.Loan + Investment.in.Mutual.Fund + Investment.Tax.Saving.Bond +
Home.Loan + Online.Purchase.Amount + status_Partner +
occupation_partner_Retired+
self_employed_partner_No + TVarea_ScottishTV + TVarea_Meridian +
gender_Female,
data=rg_train,family='binomial')

In [ ]:
summary(log.fit.final)

Now if we need to submit simple probability score we can make prediction on test data and submit that.

In [ ]:
test.prob.score= predict(log_fit,newdata = rg_test,type='response')

test.prob.score

We can save this csv file in a location. 

In [ ]:
write.csv(test.prob.score,"proper_submission_file_name.csv",row.names = F)